In [1]:
import numpy as np
import pandas as pd
## Read the data
data = pd.read_csv("application.csv") 

##  Exploratory Data Analysis

In [3]:
print(data.head(10))

   applicant_id            channel      group     city             event  \
0         10001  web-search-engine    control   Asgard  application_date   
1         10002       social-media    control  Midgard  application_date   
2         10003  web-search-engine  treatment  Midgard  application_date   
3         10004       social-media  treatment   Asgard  application_date   
4         10005       social-media  treatment  Midgard  application_date   
5         10006    job-search-site    control   Asgard  application_date   
6         10007       social-media    control  Midgard  application_date   
7         10008  web-search-engine    control   Asgard  application_date   
8         10009       social-media    control  Helheim  application_date   
9         10010    job-search-site    control   Asgard  application_date   

   event_date  
0  2018-10-01  
1  2018-10-01  
2  2018-10-01  
3  2018-10-01  
4  2018-10-01  
5  2018-10-01  
6  2018-10-01  
7  2018-10-01  
8  2018-10-01  
9  

In [33]:
# Count of each group
group_counts = data['group'].value_counts().reset_index()
group_counts.columns = ['group', 'count']

# Calculate proportion
group_counts['proportion'] = group_counts['count'] / group_counts['count'].sum()

# Format proportion as %
group_counts['proportion'] = group_counts['proportion'].apply(lambda x: f"{x:.2%}")

print(group_counts)


       group  count proportion
0    control  68750     63.46%
1  treatment  39578     36.54%


In [4]:
from scipy import stats


data['event_date'] = pd.to_datetime(data['event_date'])

events_needed = ['application_date', 'first_batch_completed_date']
filtered = data[data['event'].isin(events_needed)]


pivoted = filtered.pivot_table(index=['applicant_id', 'group'],
                               columns='event',
                               values='event_date',
                               aggfunc='first').reset_index()


pivoted = pivoted.dropna(subset=['application_date', 'first_batch_completed_date'])


pivoted['days_to_complete'] = (pivoted['first_batch_completed_date'] - pivoted['application_date']).dt.days


In [5]:
from datetime import timedelta

def mean_confidence_interval(series):
    n = len(series)
    mean = series.mean()
    std = series.std()
    se = std / np.sqrt(n)
    t_crit = stats.t.ppf(0.975, df=n-1)
    ci_margin = t_crit * se
    return pd.Series({
        'mean_days': mean,
        'ci_lower': mean - ci_margin,
        'ci_upper': mean + ci_margin,
        'n': n
    })

result = pivoted.groupby('group')['days_to_complete'].apply(mean_confidence_interval).reset_index()
print(result)

# Experiment end date
experiment_end = pd.to_datetime('2018-11-11')

# Lower bounds of 95% CI for each group
ci_dict = {
    'control': 9.9,
    'treatment': 6.8
}

       group    level_1  days_to_complete
0    control  mean_days         10.002088
1    control   ci_lower          9.897616
2    control   ci_upper         10.106561
3    control          n       2873.000000
4  treatment  mean_days          6.931607
5  treatment   ci_lower          6.838339
6  treatment   ci_upper          7.024875
7  treatment          n       2471.000000


In [6]:

# Get only application_date events
application_events = data[data['event'] == 'application_date'][['applicant_id', 'group', 'event_date']].copy()

# Compute cutoff date per group
application_events['cutoff'] = application_events['group'].map(lambda g: experiment_end - timedelta(days=ci_dict[g]))

# Find applicant_ids to keep
valid_ids = application_events[application_events['event_date'] <= application_events['cutoff']]['applicant_id'].unique()

# Keep all events for valid applicant_ids
data_cleaned = data[data['applicant_id'].isin(valid_ids)].copy()

print(f"Remaining applicants: {len(valid_ids)}")

Remaining applicants: 16224


### Problem 1:

#### To compare the result from the AB test, we can perform a Z-test with H0: conversion_rate_control = conversion_rate_treatment, and Ha: conversion_rate_control > conversion_rate_treatment.

In [9]:
# Count total applicants per group
total_applicants = data_cleaned[data_cleaned['event'] == 'application_date'].groupby('group')['applicant_id'].nunique()

# Count applicants who completed first batch per group
completed_applicants = data_cleaned[data_cleaned['event'] == 'first_batch_completed_date'].groupby('group')['applicant_id'].nunique()

# Calculate conversion rate
conversion_rate = (completed_applicants / total_applicants).reset_index()
conversion_rate.columns = ['group', 'conversion_rate']

print(conversion_rate)


       group  conversion_rate
0    control         0.263446
1  treatment         0.412767


In [10]:
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm
import numpy as np

# Get successes and total observations
successes = completed_applicants.values
nobs = total_applicants.values

# Conversion rates
p1 = successes[0] / nobs[0]  # control
p2 = successes[1] / nobs[1]  # treatment
diff = p2 - p1

# Standard error of the difference
se_diff = np.sqrt((p1 * (1 - p1) / nobs[0]) + (p2 * (1 - p2) / nobs[1]))


In [11]:
from scipy.stats import norm
import numpy as np

# Z-statistic: control - treatment
z_stat = (p2 - p1) / se_diff

# One-sided p-value for Hₐ: p_control > p_treatment
p_value = norm.sf(z_stat)

# Output
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value for Null Hyopthesis is: {p_value:.4f}")




Z-statistic: 19.2226
P-value for Null Hyopthesis is: 0.0000


In [12]:
# 95% Confidence Interval
z_critical = norm.ppf(0.975)
ci_lower = diff - z_critical * se_diff
ci_upper = diff + z_critical * se_diff

# Output
print(f"Difference in conversion rates (control - treatment): {diff:.4f}")
print(f"95% CI for the difference: [{ci_lower:.4f}, {ci_upper:.4f}]")

Difference in conversion rates (control - treatment): 0.1493
95% CI for the difference: [0.1341, 0.1645]


### Problem 2

In [39]:
# Filter applicants who initiated background check
bg_check = data_cleaned[data_cleaned['event'] == 'background_check_initiated_date'][['applicant_id', 'group']].drop_duplicates()

# Flag applicants who completed the first batch
first_batch = data_cleaned[data_cleaned['event'] == 'first_batch_completed_date'][['applicant_id']].drop_duplicates()
first_batch['completed'] = 1

# Merge and compute success flag
merged = bg_check.merge(first_batch, on='applicant_id', how='left')
merged['completed'] = merged['completed'].fillna(0).astype(int)

# Group by group and calculate metrics
conversion_summary = merged.groupby('group').agg(
    total_initiated=('applicant_id', 'nunique'),
    total_completed=('completed', 'sum')
).reset_index()

conversion_summary['conversion_rate'] = conversion_summary['total_completed'] / conversion_summary['total_initiated']

# Add cost-effectiveness columns
COST_PER_BG_CHECK = 30
conversion_summary['total_cost'] = conversion_summary['total_initiated'] * COST_PER_BG_CHECK
conversion_summary['cost_per_conversion'] = conversion_summary['total_cost'] / conversion_summary['total_completed']

# Final clean output
conversion_summary = conversion_summary[['group', 'total_initiated', 'total_completed', 'conversion_rate',
                                         'total_cost', 'cost_per_conversion']]

# Nicely formatted output
styled_summary = conversion_summary.style \
    .format({
        'total_initiated': '{:,}',
        'total_completed': '{:,}',
        'conversion_rate': '{:.2%}',
        'total_cost': '${:,.0f}',
        'cost_per_conversion': '${:,.2f}'
    }) \
    .background_gradient(subset='conversion_rate', cmap='Greens') \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'center')]
    }]) \
    .hide(axis='index')

styled_summary


group,total_initiated,total_completed,conversion_rate,total_cost,cost_per_conversion
control,"8,909","2,743",30.79%,"$267,270",$97.44
treatment,"5,812","2,399",41.28%,"$174,360",$72.68


### Problem 3

In [29]:
# Get all application_date events
applied = data_cleaned[data_cleaned['event'] == 'application_date'][['applicant_id', 'group', 'channel']].drop_duplicates()

# Get all first_batch_completed_date events
completed = data_cleaned[data_cleaned['event'] == 'first_batch_completed_date'][['applicant_id']].drop_duplicates()
completed['completed'] = 1

# Merge to check who completed
merged = applied.merge(completed, on='applicant_id', how='left')
merged['completed'] = merged['completed'].fillna(0).astype(int)

# Group by group + channel
summary = merged.groupby(['group', 'channel']).agg(
    total_applied=('applicant_id', 'nunique'),
    total_completed=('completed', 'sum')
).reset_index()

summary['conversion_rate'] = summary['total_completed'] / summary['total_applied']



       group                 channel  total_applied  total_completed  \
0    control         job-search-site           1825              284   
1    control  shopper-referral-bonus           1417              473   
2    control            social-media           3096              971   
3    control       web-search-engine           4074             1015   
4  treatment         job-search-site            965              364   
5  treatment  shopper-referral-bonus            876              416   
6  treatment            social-media           1666              616   
7  treatment       web-search-engine           2305             1003   

   conversion_rate  
0         0.155616  
1         0.333804  
2         0.313630  
3         0.249141  
4         0.377202  
5         0.474886  
6         0.369748  
7         0.435141  


In [31]:
# Pretty display with styling
styled_summary = summary.style \
    .format({
        'total_applied': '{:,}',
        'total_completed': '{:,}',
        'conversion_rate': '{:.2%}'
    }) \
    .background_gradient(subset='conversion_rate', cmap='Greens') \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'center')]
    }]) \
    .hide(axis='index')  # optional: hides the pandas row index

styled_summary


group,channel,total_applied,total_completed,conversion_rate
control,job-search-site,"1,825",284,15.56%
control,shopper-referral-bonus,"1,417",473,33.38%
control,social-media,"3,096",971,31.36%
control,web-search-engine,"4,074","1,015",24.91%
treatment,job-search-site,965,364,37.72%
treatment,shopper-referral-bonus,876,416,47.49%
treatment,social-media,"1,666",616,36.97%
treatment,web-search-engine,"2,305","1,003",43.51%
